In [46]:
import numpy as np
import matplotlib.pyplot as plt

In [111]:
class temporal_network:
    
    #node-aligned
    #get binary edge pairs
    def __init__(self, size, length, edge_list, kind = 'temporal', resolution = 1):
        
        self.nodes = [i for i in range(size)]
        self.time = length
        self.size = size
        adjacency = np.zeros((self.size*self.time,self.size*self.time))
        multi_array = np.zeros((self.time,self.size,self.size))
        
        #edge list is binary in the format of (i,j,t): 
        #there is an edge between i and j on time t(and an edge between j and i)
        for k,e in enumerate(edge_list):
            i,j,t = e[0],e[1],e[2]
            adjacency[self.size*(t-1)+i][self.size*(t-1)+j] = 1
            adjacency[self.size*(t-1)+j][self.size*(t-1)+i] = 1
            multi_array[t-1][i][j] = 1
            multi_array[t-1][j][i] = 1
            
        ##filling off diagonal blocks
        if kind == 'temporal':
            for n in range(self.size*(self.time-1)):
                adjacency[n][n+self.size] = resolution
                adjacency[n+self.size][n] = resolution
                
        elif kind == 'multilayer':
            i = 0
            while self.time-i != 0:
                i = i+1
                for n in range(self.size*(self.time-i)):
                    adjacency[n][n+i*self.size] = resolution
                    adjacency[n+i*self.size][n] = resolution
                    
        self.supra_adjacency = adjacency
        self.multi_array = multi_array
        
    def s_adjacency(self, multiarray, kind = 'temporal', resolution=1):
        #function converting a multi dimensional array into supra adjacency matrix
        n = self.size
        t = self.time
        adjacency = np.zeros((self.size*self.time,self.size*self.time))
        
        for i,c in enumerate(multiarray[:]):
            adjacency[i*n:i*n+n,i*n:i*n+n] = c     
            
        if kind == 'temporal':
            for p in range(n*(t-1)):
                adjacency[p][p+n] = resolution
                adjacency[p+n][p] = resolution
                
        elif kind == 'multilayer':
            i = 0
            while t-i != 0:
                i = i+1
                for p in range(n*(t-i)):
                    adjacency[p][p+i*n] = resolution
                    adjacency[p+i*n][p] = resolution
        return(adjacency)
        
    def aggragate(self, normalized = True):
        t = self.time 
        n = self.size
        aggragated = np.zeros((n,n))
        for i,c in enumerate(self.multi_array[:]):
            aggragated = aggragated + c
        if normalized: return (aggragated/t)
        else: return (aggragated)

In [112]:
edges = [[0,1,1],[0,1,2],[1,2,2],[0,1,3],[0,2,3],[0,1,4],[0,2,4],[1,2,4]]
T=temporal_network(3, 4, edges, kind = 'temporal', resolution = 1)
S=temporal_network(3, 4, edges, kind = 'temporal', resolution = 1).supra_adjacency
C=temporal_network(3, 4, edges, kind = 'temporal', resolution = 1).multi_array
#S,C

In [113]:
T.s_adjacency(C, kind='temporal',resolution=1)

array([[0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0.]])